In [ ]:
!wget https://cydar.ist.psu.edu/emotionchallenge/data/BOLD_ijcv.tar

--2025-03-31 05:30:44--  https://cydar.ist.psu.edu/emotionchallenge/data/BOLD_ijcv.tar
Resolving cydar.ist.psu.edu (cydar.ist.psu.edu)... 130.203.136.124
Connecting to cydar.ist.psu.edu (cydar.ist.psu.edu)|130.203.136.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15225999360 (14G) [application/x-tar]
Saving to: ‘BOLD_ijcv.tar’

BOLD_ijcv.tar       100%[===================>]  14.18G  31.8MB/s    in 6m 52s  

2025-03-31 05:37:36 (35.3 MB/s) - ‘BOLD_ijcv.tar’ saved [15225999360/15225999360]



In [ ]:
!wget https://cydar.ist.psu.edu/emotionchallenge/data/test_meta.csv

--2025-03-31 05:37:36--  https://cydar.ist.psu.edu/emotionchallenge/data/test_meta.csv
Resolving cydar.ist.psu.edu (cydar.ist.psu.edu)... 130.203.136.124
Connecting to cydar.ist.psu.edu (cydar.ist.psu.edu)|130.203.136.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 181728 (177K) [text/csv]
Saving to: ‘test_meta.csv’

test_meta.csv       100%[===================>] 177.47K   615KB/s    in 0.3s    

2025-03-31 05:37:37 (615 KB/s) - ‘test_meta.csv’ saved [181728/181728]



In [ ]:
import os
import tarfile

# Define path to the uploaded tar file and extraction directory
tar_path = "/content/BOLD_ijcv.tar"  # Replace with actual uploaded path
extract_dir = "bold_dataset/"

# Extract the tar file
if tarfile.is_tarfile(tar_path):
    with tarfile.open(tar_path, "r:*") as tar:
        tar.extractall(path=extract_dir)
        extracted_files = tar.getnames()
else:
    extracted_files = ["Provided file is not a valid tar archive."]

extracted_files[:10]  # Show a preview of extracted file structure


['BOLD_public',
 'BOLD_public/videos',
 'BOLD_public/videos/003',
 'BOLD_public/videos/003/053oq2xB3oU.mp4',
 'BOLD_public/videos/003/053oq2xB3oU.mp4/0001.mp4',
 'BOLD_public/videos/003/053oq2xB3oU.mp4/0007.mp4',
 'BOLD_public/videos/003/053oq2xB3oU.mp4/0012.mp4',
 'BOLD_public/videos/003/053oq2xB3oU.mp4/0031.mp4',
 'BOLD_public/videos/003/053oq2xB3oU.mp4/0045.mp4',
 'BOLD_public/videos/003/053oq2xB3oU.mp4/0053.mp4']

In [ ]:
import os
import numpy as np
import pandas as pd
from glob import glob
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms
import cv2
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf

In [ ]:
DATA_DIR = '/content/bold_dataset/BOLD_public'
ANNOTATION_FILE = os.path.join(DATA_DIR, 'annotations/train.csv')
NUM_CLASSES = 26
NUM_SEGMENTS = 8
BATCH_SIZE = 8
EPOCHS = 10
LR = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

Using device: cuda


In [ ]:
class BoLDDataset(Dataset):
    def __init__(self, annotation_file, video_dir, transform=None):
        self.annotations = []
        self.video_dir = video_dir
        self.transform = transform

        with open(annotation_file, "r") as f:
            lines = f.readlines()[1:]
            for line in lines:
                parts = line.strip().split(",")
                video_id = parts[0]
                person_id = parts[1]
                label_vec = parts[4]
                label = np.argmax([int(float(i)) for i in label_vec.strip().split(',')])
                self.annotations.append((video_id, person_id, label))

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        video_id, person_id, label = self.annotations[idx]
        video_path = os.path.join(self.video_dir, f"{video_id}.mp4", f"{person_id}.mp4")
        frames = self._load_video(video_path)

        if self.transform:
            frames = torch.stack([self.transform(Image.fromarray(frame)) for frame in frames])

        return frames, label

    def _load_video(self, path, num_frames=8):
      cap = cv2.VideoCapture(path)
      frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
      step = max(frame_count // num_frames, 1)

      frames = []
      for i in range(0, frame_count, step):
          cap.set(cv2.CAP_PROP_POS_FRAMES, i)
          ret, frame = cap.read()
          if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
          if len(frames) == num_frames:
            break
      cap.release()

      # ❗ Add this check
      if len(frames) == 0:
        frames = [np.zeros((224, 224, 3), dtype=np.uint8)] * num_frames
      elif len(frames) < num_frames:
        frames.extend([frames[-1]] * (num_frames - len(frames)))

      return frames


In [ ]:
class TSN(nn.Module):
    def __init__(self, num_classes=26, num_segments=8):
        super(TSN, self).__init__()
        self.num_segments = num_segments
        base_model = models.resnet18(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(base_model.children())[:-1])
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        B, T, C, H, W = x.shape
        x = x.view(-1, C, H, W)
        x = self.feature_extractor(x)
        x = x.view(B, T, -1)
        x = x.mean(dim=1)
        return self.fc(x)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
train_dataset = BoLDDataset(os.path.join(DATA_DIR, 'annotations/train.csv'), os.path.join(DATA_DIR, 'videos/003'), transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
model = TSN(num_classes=NUM_CLASSES, num_segments=NUM_SEGMENTS).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for X, y in train_loader:
        X, y = X.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        logits = model(X)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

Epoch 1, Loss: 34.2815
Epoch 2, Loss: 0.4719
Epoch 3, Loss: 0.1692
Epoch 4, Loss: 0.0775
Epoch 5, Loss: 0.0394
Epoch 6, Loss: 0.0209
Epoch 7, Loss: 0.0114
Epoch 8, Loss: 0.0063
Epoch 9, Loss: 0.0035
Epoch 10, Loss: 0.0019


In [ ]:
import onnx
#from onnx_tf.backend import prepare
import tensorflow as tf
import numpy as np

In [ ]:
model_2 = torch.load("tsn_traced.pt", weights_only=False)

/usr/local/lib/python3.11/dist-packages/torch/serialization.py:1434: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn(


In [ ]:
model_2.eval()

RecursiveScriptModule(
  original_name=TSN
  (feature_extractor): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=Conv2d)
    (1): RecursiveScriptModule(original_name=BatchNorm2d)
    (2): RecursiveScriptModule(original_name=ReLU)
    (3): RecursiveScriptModule(original_name=MaxPool2d)
    (4): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(
        original_name=BasicBlock
        (conv1): RecursiveScriptModule(original_name=Conv2d)
        (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
        (relu): RecursiveScriptModule(original_name=ReLU)
        (conv2): RecursiveScriptModule(original_name=Conv2d)
        (bn2): RecursiveScriptModule(original_name=BatchNorm2d)
      )
      (1): RecursiveScriptModule(
        original_name=BasicBlock
        (conv1): RecursiveScriptModule(original_name=Conv2d)
        (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
        (relu): RecursiveS

In [ ]:
!pip install keras.src.engine

ERROR: Could not find a version that satisfies the requirement keras.src.engine (from versions: none)
ERROR: No matching distribution found for keras.src.engine
